In [43]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, Reshape
from tensorflow.keras.optimizers import Adam, RMSprop

<center><h2>Jupyter Notebook para entender el modelamiento matemático de las GANs y ponerlo en práctica.</h2></center><br>
<b>Autor:</b> Edwin Jahir Rueda Rojas<br>
<b>email:</b> ejrueda95g@gmail.com<br>
<b>website:</b> edwinrueda.com<br>
<b>github:</b> https://github.com/ejrueda<br><br>
Todo el contenido aquí realizado se basa en el artículo publicado por Ian J. Goodfellow et al. Titulado: <i><b>Generative Adversarial Nets</b></i>. (https://arxiv.org/abs/1406.2661)

<h1>Arquitectura GAN</h1><br>
- Se entrenan dos modelos G y D simultaneamente, en los cuales el modelo generativo G captura la distribuición de los datos reales, y el modelo discriminativo D estima la probabilidad de que dado un ejemplo, este sea de la distribuición original o no.

#### Objetivo:
- El objetivo de esta red es entrenar la red generadora G para maximizar la probabilidad de que la red Discriminadora D cometa errores de intendificación de la distribuición. Esto corresponde a un juego de mini-max entre dos jugadores, y solo existe una única solución, la cual ocurre cuando la red Generadora G capta la distribuición de los datos y así la red Discriminadora D no consigue diferenciar los ejemplos, teniendo así 0.5 de acierto.

Cabe aclarar que esto se puede implementar con dos redes perceptrones multicapa utilizando el algoritmo <i>backpropagation</i> para el entrenamiento.

## Ejemplo pŕactico
- Para nuestro ejemplo práctico, se tomará tomará una base de datos de genes y se trataŕa de generar nuevos genes a partir del modelamiento de la distribuición que aprenderá la red Generadora.

### Red Generadora G
- Se toma una distribuición Gaussiana como entrada a la red Generadora para así generar los datos sintéticos (fake).

In [89]:
def generator(noise_input, out_dim):
    """
    noise_input: vector con los datos de la distribuición inicial
    out_dim: dimension de la salida esperada, por ejemplo si es una imagen
             en escala de gris, sería out_dim: (32,32,1)
    Retorna el modelo Generador
    """
    model = Sequential()
    model.add(Dense(256, input_shape=(noise_input,)))
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(.1))
    model.add(Dense(np.prod(out_dim), activation="tanh"))
    model.add(Reshape(out_dim))
    
    return model

In [90]:
noise_input = 100
print("Dimension del ruido de entrada: ",noise_input)
G = generator(noise_input, out_dim=(9,))
optimizerG = Adam(lr=0.0004, beta_1=0.5)
G.compile(loss='binary_crossentropy', optimizer=optimizerG)

Dimension del ruido de entrada:  100


In [91]:
G.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 9)                 2313      
_________________________________________________________________
reshape_11 (Reshape)         (None, 9)                 0         
Total params: 28,169
Trainable params: 28,169
Non-trainable params: 0
_________________________________________________________________


In [93]:
noise = np.random.normal(loc=0, scale=1, size=(50,100))
print("Dimension de los datos de entrada:", noise.shape)
data_fake = G.predict(noise)
print("Dimension de los ejemplos generados:", data_fake.shape)

Dimension de los datos de entrada: (50, 100)
Dimension de los ejemplos generados: (50, 9)
